# Segmenting neighborhoods in Toronto



Import needed dependencies.


In [1]:
#Web scrapping stage
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

import lxml
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>


## [1. Get Toronto's neighborhoods](#get_neighborhoods)

The Wikipedia article containing Toronto's neighborhoods is: https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto

From there I need to get the names.

To download the contents of the webpage `url` in text format and store in a variable called `data` and create a `BeautifulSoup` object using the BeautifulSoup constructor:

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'
#soup

**Explore the data**

Table with district numbers.

In [3]:
table = soup.find_all("table",{"class":"wikitable sortable"})
table = table[1]
#table

Get table rows' attributes to a DataFrame:

In [4]:
df = pd.read_html(str(table), flavor='html5lib')
df = df[0]
#df.head()

In [5]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
def get_postcode(neighbourhood, borough):
    address = str(neighbourhood)+", "+borough+',Toronto, Ontario, Canada'
    geolocator = Nominatim(user_agent="to_explorer")
    location = geolocator.geocode(address)
    #print(location)
    if location != None:
        postcode = str(location).split(',')[-2]
        #print(postcode)
        postcode = postcode.split(" ")[1]
        #print(postcode)
        latitude = location.latitude
        longitude = location.longitude
        if postcode != "Ontario":
            return [postcode,latitude,longitude]

In [7]:
for row in df.values:
    #print(row)
    borough = str(row[2])
    neighbourhood = str(row[1])
    #rint(borough+" "+neighbourhood)
    
    #Get latitude and longitude
    try:
        [postcode,latitude,longitude] = get_postcode(neighbourhood, borough)
        neighborhoods = neighborhoods.append({'PostalCode': postcode,
                                            'Borough': borough,
                                            'Neighborhood': neighbourhood,
                                            'Latitude': latitude,
                                            'Longitude': longitude}, ignore_index=True)
    except:
        continue

In [8]:
neighborhoods.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1V,Scarborough,Agincourt North,43.808038,-79.266439
1,M1S,Scarborough,Agincourt South-Malvern West,43.781969,-79.257689
2,M8W,Etobicoke,Alderwood,43.601717,-79.545232
3,M5T,Old City of Toronto,Annex,43.670338,-79.407117
4,M2L,North York,Banbury-Don Mills,43.752339,-79.365716
5,M5R,Old City of Toronto,Bay Street Corridor,43.672135,-79.390499
6,M2K,North York,Bayview Village,43.769197,-79.376662
7,M2M,North York,Bayview Woods-Steeles,43.798127,-79.382973
8,M1P,Scarborough,Bendale,43.753520,-79.255336
9,M6L,North York,Black Creek,43.695400,-79.485495
